1. So, I need a comprehensive set of movies and all the people that worked for it. My way to do it would be to split the df vertically into citing movie info df and cited movie df.

2. Then concatenate those, check for doubles and check if they have the same people working on the team or there are discrepancies.

3. Finally cancel replicates of movies, rows that are double basically.

In [1]:
import os
import pandas as pd

# creating the tables with pandas to interact with our datasets

#name_basics = pd.read_csv("../Dataset/name.basics.tsv" , sep='\t', header=0)
crew_data = pd.read_csv("../data/filtered_graph2_data.csv" , header=0)

citing_movies = crew_data[["tconst_movie", "movie", "movie_country", "year", "titleType", "tconst_cit", "nconst_movie", "category_movie", "job_movie"]]
cited_movies = crew_data[["tconst_cit", "cit", "country_cit", "primaryTitle_cit", "year_cit", "titleType_cit", "nconst_cit", "category_cit", "job_cit"]]
'''citing_countries = set(citing_movies["movie_country"])
cited_countries = set(cited_movies["country_cit"])
print(citing_countries) #7
print(cited_countries) #7'''

column_names = list(crew_data.columns.values)
#name_basics[:30]
#print(set(crew_data.category_cit.values))
#citing_movies.describe(include="all")
#citing_movies.dtypes

# starting to filter dataset
citing_movies = citing_movies[citing_movies['titleType'] != 'short']
citing_movies['year'] = citing_movies['year'].astype("string")

#function to remove .0 in year column
def remove_dotzero(s):
    return s.replace(".0", "")

citing_movies["year"] = citing_movies["year"].apply(remove_dotzero)

#renaming the columns for mental order
#N.B. because I put inplace = True I don't need to reassing the renamed columns to a new df variable, the original df is being modified on the fly.
citing_movies.rename(columns = {'tconst_movie' : 'movie_ID', 'movie' : 'title', 'nconst_movie' : 'person_ID', 'category_movie'  : 'person_role', 'tconst_cit' : 'cites'}, inplace= True)

#drop titleType column
#citing_movies = citing_movies.drop(['titleType', 'job_movie'], axis=1)

#analysing the ds 
'''temp =  citing_movies.query("movie_ID == 'tt5611424'")
print(len(temp))
temp
'''
#drop titleType column
citing_movies = citing_movies.drop(['titleType', 'job_movie'], axis=1)

citing_countries = set(citing_movies["movie_country"])
print(citing_countries)

citing_movies

{'Hong Kong', 'Japan', 'USA', 'China', 'Taiwan', 'Mongolia', 'South Korea'}


,movie_ID,title,movie_country,year,cites,person_ID,person_role
0,tt5611424,#FromJennifer (2017),USA,2017,tt4627224,nm3106201,director
1,tt5611424,#FromJennifer (2017),USA,2017,tt4627224,nm3106201,director
2,tt5611424,#FromJennifer (2017),USA,2017,tt4627224,nm3106201,director
3,tt5611424,#FromJennifer (2017),USA,2017,tt4627224,nm3106201,director
4,tt5611424,#FromJennifer (2017),USA,2017,tt4627224,nm4097598,writer
...,...,...,...,...,...,...,...
1449957,tt6565702,X-Men: Dark Phoenix (2018),USA,2018,tt1270798,nm0795682,producer
1449958,tt6565702,X-Men: Dark Phoenix (2018),USA,2018,tt1270798,nm0795682,producer
1449959,tt6565702,X-Men: Dark Phoenix (2018),USA,2018,tt1270798,nm0795682,producer
1449960,tt6565702,X-Men: Dark Phoenix (2018),USA,2018,tt1270798,nm0795682,producer


I realize that the reason why the same movie is repeated so many times is because for every movie also the info about the movie they cite. So for every movie cited there will be three lines for the cited movie director, writer and producer.

I will just remove duplicate rows in the df

In [2]:
print(len(citing_movies))
citing_movies = citing_movies.drop_duplicates()
print(len(citing_movies))

citing_movies

1449962
346642


,movie_ID,title,movie_country,year,cites,person_ID,person_role
0,tt5611424,#FromJennifer (2017),USA,2017,tt4627224,nm3106201,director
4,tt5611424,#FromJennifer (2017),USA,2017,tt4627224,nm4097598,writer
8,tt5611424,#FromJennifer (2017),USA,2017,tt4627224,nm3359084,producer
12,tt5611424,#FromJennifer (2017),USA,2017,tt2630300,nm3106201,director
13,tt5611424,#FromJennifer (2017),USA,2017,tt2630300,nm4097598,writer
...,...,...,...,...,...,...,...
1449932,tt6565702,X-Men: Dark Phoenix (2018),USA,2018,tt0376994,nm0795682,producer
1449938,tt6565702,X-Men: Dark Phoenix (2018),USA,2018,tt1270798,nm1334526,director
1449944,tt6565702,X-Men: Dark Phoenix (2018),USA,2018,tt1270798,nm0356745,producer
1449950,tt6565702,X-Men: Dark Phoenix (2018),USA,2018,tt1270798,nm0404446,producer


In [3]:
#additional exploring of the df
mostCiting =  citing_movies.query("movie_ID == 'tt5611424'")
print(len(mostCiting))
mostCiting = mostCiting.drop_duplicates()
print(len(mostCiting))
mostCiting.describe(include="all")

citing_movies.describe(include="all")
mostCiting.describe(include="all")


6
6


,movie_ID,title,movie_country,year,cites,person_ID,person_role
count,6,6,6,6,6,6,6
unique,1,1,1,1,2,3,3
top,tt5611424,#FromJennifer (2017),USA,2017,tt4627224,nm3106201,director
freq,6,6,6,6,3,2,2


In [4]:
from pprint import pprint

# iterating over columns unique values to see what values are repeated
column_names = list(mostCiting.columns.values) #put headers in a list
print(column_names)
'''print(len(mostCiting.cites.values))
print(len(set(mostCiting.cites.values)))'''

i = 0
while i <= len(column_names)-1:
    print("\n\n all values of the " + column_names[i] + " column are: \n")
    print(mostCiting[column_names[i]].values)
    print(set(mostCiting[column_names[i]].values))
    print(len(set(mostCiting[column_names[i]].values)))
 
    i += 1

['movie_ID', 'title', 'movie_country', 'year', 'cites', 'person_ID', 'person_role']


 all values of the movie_ID column are: 

['tt5611424' 'tt5611424' 'tt5611424' 'tt5611424' 'tt5611424' 'tt5611424']
{'tt5611424'}
1


 all values of the title column are: 

['#FromJennifer (2017)' '#FromJennifer (2017)' '#FromJennifer (2017)'
 '#FromJennifer (2017)' '#FromJennifer (2017)' '#FromJennifer (2017)']
{'#FromJennifer (2017)'}
1


 all values of the movie_country column are: 

['USA' 'USA' 'USA' 'USA' 'USA' 'USA']
{'USA'}
1


 all values of the year column are: 

['2017' '2017' '2017' '2017' '2017' '2017']
{'2017'}
1


 all values of the cites column are: 

['tt4627224' 'tt4627224' 'tt4627224' 'tt2630300' 'tt2630300' 'tt2630300']
{'tt2630300', 'tt4627224'}
2


 all values of the person_ID column are: 

['nm3106201' 'nm4097598' 'nm3359084' 'nm3106201' 'nm4097598' 'nm3359084']
{'nm3359084', 'nm4097598', 'nm3106201'}
3


 all values of the person_role column are: 

['director' 'writer' 'produce

So, repetitions occur because of:
1. members of crew
2. multiple countries of production

with Be kind rewind, the most frequent movie in our dataframe, whe have:

537 citations * 3 crew members * 3 countries of production = 4833 instances

### Now I proceed to clean the cited movies dataframe:

In [5]:
cited_movies = crew_data[["tconst_cit", "cit", "country_cit", "primaryTitle_cit", "year_cit", "titleType_cit", "nconst_cit", "category_cit", "job_cit"]]

# starting to filter dataset
cited_movies = cited_movies[cited_movies['titleType_cit'] != 'short']
cited_movies['year_cit'] = cited_movies['year_cit'].astype("string")

#function to remove .0 in year column
def remove_dotzero(s):
    return s.replace(".0", "")

cited_movies["year_cit"] = cited_movies["year_cit"].apply(remove_dotzero)

#renaming the columns as the citing_movie df
cited_movies.rename(columns = {'tconst_cit' : 'movie_ID', 'cit' : 'title', 'country_cit' : 'movie_country', 'year_cit' : 'year', 'nconst_cit' : 'person_ID', 'category_cit'  : 'person_role'}, inplace= True)

#N.B this df has no 'cite' column: for the movies that are already present in citing_movies, I can just drop the rows that are repeated (drop duplicates, keep first instance).
# but there might be cited movies that cite no film
#  

#drop titleType column
cited_movies = cited_movies.drop(['titleType_cit', 'job_cit', 'primaryTitle_cit'], axis=1)
print(len(cited_movies))
cited_movies = cited_movies.drop_duplicates()
print(len(cited_movies))

cited_countries = set(cited_movies["movie_country"])
print(citing_countries)

cited_movies

1449962
65120
{'Hong Kong', 'Japan', 'USA', 'China', 'Taiwan', 'Mongolia', 'South Korea'}


,movie_ID,title,movie_country,year,person_ID,person_role
0,tt4627224,2 Jennifer (2016),USA,2016,nm4097598,writer
1,tt4627224,2 Jennifer (2016),USA,2016,nm1735659,producer
2,tt4627224,2 Jennifer (2016),USA,2016,nm4259291,producer
3,tt4627224,2 Jennifer (2016),USA,2016,nm3106201,producer
12,tt2630300,To Jennifer (2013),USA,2013,NaN,NaN
...,...,...,...,...,...,...
1449735,tt3315342,Logan (2017),USA,2017,nm0003506,director
1449736,tt3315342,Logan (2017),USA,2017,nm0291082,writer
1449737,tt3315342,Logan (2017),USA,2017,nm0338169,writer
1449738,tt3315342,Logan (2017),USA,2017,nm0739661,writer


In [6]:
cited_movies.describe(include="all")

,movie_ID,title,movie_country,year,person_ID,person_role
count,65120,65120,65120,65120,64733,64733
unique,17309,16909,7,114,24057,3
top,tt0425637,The Revenant (2015),USA,2006,nm0297772,writer
freq,30,33,54887,1153,88,30726


Now I need to find the intersection between citing_movies id and cited_movies.
I need to:
1. find out what movies in cited_movies are already present in citing_movies
2. check wether the information for same movie_ID are coherent
3. if they are, they are duplicates and I can just drop the cited_movie instance
4. else, inquire why the related info are different and eventually assing the cites value to the cited_movie instance.
5. for movies only present in the cited_movie df, leave them in the concatenated df with nan value in cites column

In [7]:
# First I want to save data about citations from the first dataframe
# I will do it just by creating a subdataframe with the relevant columns

citations_data = citing_movies[["movie_ID", "cites"]]
citations_data

,movie_ID,cites
0,tt5611424,tt4627224
4,tt5611424,tt4627224
8,tt5611424,tt4627224
12,tt5611424,tt2630300
13,tt5611424,tt2630300
...,...,...
1449932,tt6565702,tt0376994
1449938,tt6565702,tt1270798
1449944,tt6565702,tt1270798
1449950,tt6565702,tt1270798


In [8]:
# there are 16379 citing movies, 21453 cited movies and 5349 are in common but ofc in the dataframes are repeated
# Than I add cites column to cited movies
cited_movies.insert(4, 'cites', pd.Series(dtype="string"))

# concatenating
concat_df = pd.concat([citing_movies, cited_movies])
#len(concat_df) #433829
concat_df.drop_duplicates(inplace=True)
#subset=['movie_ID', 'title', 'movie_country', 'year', 'person_ID', 'person_role']

len(concat_df) #142818
cited_countries = set(concat_df["movie_country"])
print(citing_countries)


{'Hong Kong', 'Japan', 'USA', 'China', 'Taiwan', 'Mongolia', 'South Korea'}


Now I need to add the info about the people that I can retreive from name.basics.csv

In [20]:
people_data = pd.read_csv("../data/name.basics.tsv", sep="\t", header=0)
people_data = people_data[["nconst", "primaryName", "birthYear", "deathYear"]]
people_data

network2data = concat_df.merge(people_data, left_on="person_ID", right_on="nconst", how="left") #should be 142818 rows still
network2data.to_csv("../data/network2data.csv")
